In [22]:
%matplotlib notebook
from matplotlib import pyplot as plt
import json
from collections import Counter
from datetime import datetime

In [23]:
t1 = datetime.now()
s = ""
with open("yelp_dataset_challenge_round9/yelp_academic_dataset_review.json") as f:
    reviews = f.read().strip().split("\n")
print(datetime.now())
print(len(reviews))

2017-04-09 23:53:07.320384
4153150


In [72]:
#Print the first review
print(reviews[0])
print("\n"+reviews[1])
print("\n"+reviews[10000])

{"review_id":"NxL8SIC5yqOdnlXCg18IBg","user_id":"KpkOkG6RIf4Ra25Lhhxf1A","business_id":"2aFiy99vNLklCx3T_tGS9A","stars":5,"date":"2011-10-10","text":"If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly. The time he has spent here has been very productive and working with him educational and enjoyable. I hope not to need him again (though this is highly unlikely) but knowing he is there if I do is very nice. By the way, I'm not from El Centro, CA. but Scottsdale, AZ.","useful":0,"funny":0,"cool":0,"type":"review"}

{"review_id":"pXbbIgOXvLuTi_SPs1hQEQ","user_id":"bQ7fQq1otn9hKX-gXRsrgA","business_id":"2aFiy99vNLklCx3T_tGS9A","stars":5,"date":"2010-12-29","text":"After being on the phone with Verizon Wireless trying to figure out why my phone wasn't working for 4.5 hours, I was put in touch with Sharpie Tech. Well, after 10 seconds they fixed the problem. As the owner of a company that does our best numbers over the holiday seaso

In [80]:
from nltk.tokenize import sent_tokenize
import json
first_review = json.loads(reviews[0])
second_review = json.loads(reviews[1])
nth_review = json.loads(reviews[10000])


TypeError: expected string or buffer

In [81]:
review_text = first_review['text']
review_text2 = second_review['text']
review_text3 = nth_review['text']

In [82]:

sent_tokenize(review_text)

[u'If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly.',
 u'The time he has spent here has been very productive and working with him educational and enjoyable.',
 u'I hope not to need him again (though this is highly unlikely) but knowing he is there if I do is very nice.',
 u"By the way, I'm not from El Centro, CA.",
 u'but Scottsdale, AZ.']

In [83]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
tokenized_text = word_tokenize(review_text)
#remove stop words
english_stops = set(stopwords.words('english'))
#text after stop words are removed
words = [word for word in tokenized_text if word not in english_stops]
#print(words)
#turns out that we probably do not want to remove stop words, as neutrality is measured in 

In [84]:
from nltk.corpus import subjectivity
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:100]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:100]]
#print(subj_docs)
#print(obj_docs)

In [85]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:100]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:100]]

#We separately split subjective and objective instances to keep a balanced uniform class distribution in both train and test sets.

train_subj_docs = subj_docs[:80]
test_subj_docs = subj_docs[80:100]
train_obj_docs = obj_docs[:80]
test_obj_docs = obj_docs[80:100]
training_docs = train_subj_docs+train_obj_docs
testing_docs = test_subj_docs+test_obj_docs
sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)

#print(all_words_neg)
#print("\n!!!")
#print(unigram_feats)
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)
# will most likely not use this but just use the vader sentinment Intensity analysis instead

In [86]:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Training classifier
Evaluating NaiveBayesClassifier results...
Accuracy: 0.8
F-measure [obj]: 0.8
F-measure [subj]: 0.8
Precision [obj]: 0.8
Precision [subj]: 0.8
Recall [obj]: 0.8
Recall [subj]: 0.8


In [94]:
# Sentiment Intensity Analyzer Using Vader
from __future__ import print_function
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
reviews = [first_review, second_review, nth_review]
sid = SentimentIntensityAnalyzer()
for review in reviews:
    review_text = review['text']
    rating = review['stars']
    ss = sid.polarity_scores(review_text)
    print(review_text + "\n\nActual rating: {}".format(rating))
    for k in sorted(ss):
        print("{0}:{1} ".format(k, ss[k]),end='')
    print("\n\n")

If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly. The time he has spent here has been very productive and working with him educational and enjoyable. I hope not to need him again (though this is highly unlikely) but knowing he is there if I do is very nice. By the way, I'm not from El Centro, CA. but Scottsdale, AZ.

Actual rating: 5
compound:0.8895 neg:0.0 neu:0.823 pos:0.177 


After being on the phone with Verizon Wireless trying to figure out why my phone wasn't working for 4.5 hours, I was put in touch with Sharpie Tech. Well, after 10 seconds they fixed the problem. As the owner of a company that does our best numbers over the holiday season, having my phone out of order for 4.5 hours was horrible. Sharpie Tech fixed the problem in record time, even Verizon was shocked. I highly recommend working with this company and I look forward to working with them more. 

-Rachel Charlupski
Founder and CEO, The Babysitting Compan